In [1]:
from dabry.problem import NavigationProblem, IndexedProblem, DatabaseProblem
from dabry.misc import Utils
from dabry.wind import *
from dabry.model import *
from dabry.penalty import *
from dabry.solver_ef import SolverEF
from dabry.ddf_manager import DDFmanager

In [2]:
pb = IndexedProblem('linear')

f = 1e6
c = f* np.array((0.5, 0.17))
pmax = 1000
rten = f* 0.2
sigma = rten / (2 * np.log(10))

def pf(x):
    return 1 + pmax*np.exp(-0.5*(x - c) @ (x - c)/sigma**2)

penalty = Penalty(pf, length_scale=f)

pb.penalty = penalty

In [39]:
x_init, x_target, start_date, airspeed, level = \
Utils.process_pb_params(*'-16.44 13.32 -35.07 -5.8 202111051200 23.0 1000'.split(' '))
duration = 2 * Utils.distance(Utils.DEG_TO_RAD * x_init, Utils.DEG_TO_RAD * x_target,
                                      coords=Utils.COORD_GCS) / airspeed
stop_date = datetime.fromtimestamp(start_date.timestamp() + duration)
pb = DatabaseProblem(x_init=Utils.DEG_TO_RAD * x_init,
                             x_target=Utils.DEG_TO_RAD * x_target, airspeed=airspeed,
                             t_start=start_date.timestamp(), t_end=stop_date.timestamp(),
                             altitude=level,
                             resolution='0.5')

 26%|███████████                                | 10/39 [00:00<00:01, 18.76it/s]

In [40]:
solver = SolverEF(pb, max_steps=500, max_time=50*3600)

In [41]:
res = solver.solve()

In [42]:
ddf = DDFmanager()
ddf.setup()
ddf.set_case('test_backward')
ddf.clean_output_dir()
ddf.dump_wind(pb.model.wind, nx=101, ny=101, bl=pb.bl, tr=pb.tr)
ddf.dump_trajs(solver.get_trajs(dual_only=True))
ddf.dump_trajs([res.traj], filename='trajectories_timeopt.h5')
ddf.log(pb)

In [25]:
fb = solver.get_control_map(nx=71, ny=71)

In [47]:
pb.load_feedback(fb)
from dabry.stoppingcond import DistanceSC
sc = DistanceSC(lambda x: (x - pb.x_target) @ (x - pb.x_target), 0.0001)
traj = pb.integrate_trajectory(pb.x_init + np.array((-0.2, 0)), sc, int_step=200, t_init=start_date.timestamp())
ddf.dump_trajs([traj])

In [21]:
traj.timestamps[traj.last_index]

1635884700.0

In [23]:
np.all(fb.values < 1e-6)

False

In [15]:
datetime.fromtimestamp(solver.ef_dual.trajs[0].particles[1].t)

datetime.datetime(2021, 11, 1, 11, 37, 25, 247086)